### 新建电影库和标签库

In [2]:
import pandas as pd

mtag=pd.DataFrame(columns=list(['标签','链接地址']))
mdata=pd.DataFrame(columns=list(['名称','年份','评分','评论数','影评数','标签','链接地址']))

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

'''
获取详情相关数据
'''
def get_movie_details(url):
    try:
        mhtml=requests.get(url,headers=headers)
        soup=BeautifulSoup(mhtml.text,'html.parser')
        '''
        名称
        '''
        name= soup.find('span',property="v:itemreviewed").text
        year= soup.find('span',class_="year").text.replace('(','').replace(')','')
        score =soup.find('strong',class_="ll rating_num").text
        commentCount=soup.find('a',href=url+"comments?status=P").text.replace(' ','').replace('全部','').replace('条','')
         
        tag=div_tag=soup.find('div',class_='tags-body').text

        reviewCount=soup.find_all('a',href="reviews")[0].text.replace(' ','').replace('全部','').replace('条','')
        # return {'名称':name,'年份':year,'评分':score,'评论数':commentCount,'影评数':reviewCount,'链接地址':url}
        return {'名称': name, '年份': year, '评分': score, '评论数': commentCount, '影评数': reviewCount,'标签':tag,'链接地址':url}
    except(Exception) as err:
        print(err)
    
    return {'名称': '', '年份': '', '评分': '', '评论数': '', '影评数': '','标签':'','链接地址':url}

In [6]:
get_movie_details('https://movie.douban.com/subject/27119724/')

{'名称': '小丑 Joker',
 '年份': '2019',
 '评分': '8.7',
 '评论数': '198464',
 '影评数': '4903',
 '标签': '\n人性\n犯罪\nDC\n心理\n社会\n美国\n2019\n暴力\n',
 '链接地址': 'https://movie.douban.com/subject/27119724/'}

In [ ]:
import json

flag=True
pageIndxe=0
while(flag):
    mlist=requests.get('https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&sort=rank&page_limit=20&page_start='+str(pageIndxe),headers=headers)
    mdict = json.loads(mlist.text)
    mplist=mdict["subjects"]
    if len(mplist)==0:
        flag=False
    pageIndxe=pageIndxe+20

    for mp in mplist:
        m=get_movie_details(mp["url"])
        # 添加电影数据
        mdata.loc[len(mdata)]=m
        
        # 添加标签数据
        for tmp_tag in m['标签'].split('\n'):
            add_tag(tmp_tag,mp["url"])
    

        print("采集中："+mp["url"])
print("完成") 